<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/myGPTlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello World App

This is a Hello World App. Some words about it. Lorem ipsum.

# Setup and Configuration

In [ ]:
# @title Settings

# @markdown Default model
default_model = 'GPT-3.5' # @param ["GPT-3.5", "GPT-4"]

# @markdown Initial Setup Mode for pip install, fetch credentials etc.
initial_setup_mode = True # @param {type:"boolean"}

# @markdown Debug Mode for extensive logging.
debug_mode = True # @param {type:"boolean"}

# @markdown iOS Mode to develop helper functions, no Gradio.
# @markdown Useful for development on iOS, eg. with Carnets App
ios_mode = False # @param {type:"boolean"}

In [ ]:
if ios_mode == False:
  print('Mac')
else:
  print('iOS')

In [ ]:
if initial_setup_mode == True:
  !pip install openai==0.27.7 yt-dlp==2023.7.6 librosa==0.10.0.post2 pickle-mixin==1.0.2 langchain==0.0.225 PyPDF2==3.0.1 PyMuPDF==1.22.5 -q
else:
  print('No initial setup.')


In [ ]:
if (initial_setup_mode == True) and (ios_mode == False) :
  !pip install gradio -q
else:
  print('No initial setup / iOS.')

In [ ]:
if (initial_setup_mode == True) and (ios_mode == False) :
  %load_ext gradio
else:
  print('No initial setup / iOS.')

In [ ]:
if initial_setup_mode == True:
  !wget -q bit.ly/aknip-colab-setup
  %run aknip-colab-setup
else:
  print('No initial setup.')

Secrets (JSON string): ··········
Credentials stored in env 'CREDS'. Get values via: 
creds = json.loads(os.getenv('CREDS')) 
key = creds['OpenAI']['v1']['credential']


In [ ]:
creds = json.loads(os.getenv('CREDS'))
key = creds['OpenAI']['v2']['credential']
# print(key)

# Helper Functions

In [ ]:
# Some Code

# The App

In [54]:
if ios_mode == False:
  import gradio as gr

  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )

  css = """
    .gradio-container {max-width: 700px!important}
    .vspacer1 {margin-top: 50px}
  """


  def greet(name):
    result = "HALLO " + name + "!!!"
    return result

  def save(text):
    result = "HALLO " + text + "!!!"
    return result

  def upload_file(my_file):
    import tempfile
    # Save the uploaded file to a temporary location
    with tempfile.NamedTemporaryFile(delete=False, suffix=".txt") as temp_pdf_file:
        temp_pdf_file.write(my_file.read())
    debug = "my: " + my_file.name + "\ntmp: " + temp_pdf_file.name

    the_file = my_file.name
    f= open(the_file,'r')
    if f.mode == 'r':
          contents =f.read()
          debug = debug + "\n inside... "
          debug = debug + "\n content = " + contents
    f.close()


    return debug



  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# ChatGPTLab 2.0", elem_classes="vspacer1")
      gr.Markdown("### Optimizing your work with LLMs.")

      def file_name_reader(file):
        file_name = file.name  # extract the file name from the uploaded file
        return file_name

      def func2(name, file):
        file_name = name
        file_x = file
        return "yes, done!"

      with gr.Tab("Input Text 2"):
        gr.Markdown("Please enter text")
        file_obj = gr.File(label="Input File", file_count="single", file_types=[".txt",".md"])
        # extract the filename from gradio.io file object
        keyfile_name = gr.Interface(file_name_reader, inputs="file", outputs=None)
        tab2_inputs = [keyfile_name, file_obj]
        tab2_outputs = gr.Textbox(label="Text output")
        tab2_submit_button = gr.Button("Submit")
        tab2_submit_button.click(func2, inputs=tab2_inputs, outputs=tab2_outputs)

      with gr.Tab("Input Text 1"):
        gr.Markdown("Please enter text")
        #file_output = gr.File()
        file_output = gr.Textbox(label="Text output")
        #upload_button = gr.UploadButton("Click to Upload a File", file_types=[".txt",".md"], file_count="single")
        upload_button = gr.File()
        upload_button.upload(upload_file, upload_button, file_output)


      with gr.Tab("Step 1"):
        gr.Markdown("Please enter all the data which is needed for this application:")
        text_input = gr.Textbox(label="Filename", value="hello")
        text_output = gr.Textbox(label="Text output")
        text_button = gr.Button("Save")
        text_button.click(save, text_input, text_output, api_name="save")

      with gr.Tab("Step 2"):
        gr.Markdown("Please select the optimization:")
        radio = gr.Radio(
          ["by headline", "by paragraph", "by §§§"], label="Text split method"
        )
        name = gr.Textbox(label="Name", placeholder="Enter text...")
        output = gr.Textbox(label="Output Box")
        greet_btn = gr.Button("Start", scale=0)
        greet_btn.click(fn=greet, inputs=name, outputs=output, api_name="greet")

  demo.launch(quiet=True, share=False)

else:
  print('iOS Mode - Nothing to do.')

<IPython.core.display.Javascript object>

In [ ]:


# https://stackoverflow.com/questions/75458603/how-to-output-downloadable-file-after-processing
# https://stackoverflow.com/questions/76048858/how-to-fix-pypdf2-not-reading-uploading-pdfs-through-gradio-issue
# https://stackoverflow.com/questions/76083621/how-to-get-the-upload-file-location-in-gradio


import os
import shutil

the_file = "/tmp/tmptw7ihlpx.txt"

f= open(the_file,'r')
if f.mode == 'r':
      contents =f.read()
      print('inside')
      print(contents)
f.close()

shutil.copyfile(the_file, "./yoyo.txt")

inside



'./yoyo.txt'

In [ ]:
!pip install gTTS==2.2.1 pdfminer.six==20201018

In [ ]:
# https://github.com/salma71/text_speech

from gtts import gTTS
from pdfminer.high_level import extract_text
import gradio as gr
import os


def pdf_to_text(file_obj):
    text = extract_text(file_obj.name)
    #myobj = gTTS(text=text, lang='en', slow=False)
    #myobj.save("test.wav")
    return 'test.wav'


examples = [
    [os.path.abspath("short-pdf.pdf")],
    [os.path.abspath("long-pdf.pdf")]
]


iface = gr.Interface(fn = pdf_to_text,
                     inputs = 'file',
                     outputs = 'audio',
                     verbose = True,
                     title = 'PDF to Audio Application',
                     description = 'A simple application to convert PDF files in audio speech. Upload your own file, or click one of the examples to load them.',
                     article =
                        '''<div>
                            <p style="text-align: center"> All you need to do is to upload the pdf file and hit submit, then wait for compiling. After that click on Play/Pause for listing to the audio. The audio is saved in a wav format.</p>
                        </div>''',
                     examples=examples
                    )

iface.launch()

<ipython-input-51-1a6486f12c23>:20: GradioDeprecationWarning: `verbose` parameter is deprecated, and it has no effect
  iface = gr.Interface(fn = pdf_to_text,


FileNotFoundError: ignored